## Modulo designado para la creación de el modelo de Random Forest
En este punto los datasets ya se encuentran en archivos .csv dentro de la carpeta
"CSV ready to model"


In [7]:
import pandas as pd
from scikitplot.metrics import plot_roc, plot_roc_curve
from sklearn.preprocessing import LabelEncoder
import numpy as np
import scikitplot as skplt#Esta librería toca instalarla "conda install -c conda-forge scikit-plot"
import time
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score, roc_auc_score, roc_curve
df=pd.read_csv('../../CSV ready to model/withoutPCA.csv')

Label encoder para la columna a predecir

In [8]:
df['GRAVEDAD'].value_counts()

HERIDO        60201
SOLO DAÑOS    52790
MUERTO          668
Name: GRAVEDAD, dtype: int64

In [9]:
le=LabelEncoder()
df['GRAVEDAD']=df.loc[:,['GRAVEDAD']].apply(le.fit_transform)

In [10]:
train,test = [x for _, x in df.groupby(df['PERIODO'] > 2018)]

In [11]:
train=train.drop(['PERIODO'],axis=1)
test=test.drop(['PERIODO'],axis=1)
y_train = train['GRAVEDAD'].values
x_train = (train.drop(['GRAVEDAD'], axis = 1)).values
y_test = test['GRAVEDAD'].values
x_test=(test.drop(['GRAVEDAD'],axis=1)).values


In [12]:

def classification_error(y_est, y_real):
    err = 0
    for y_e, y_r in zip(y_est, y_real):

        if y_e != y_r:
            err += 1

    return err/np.size(y_est)


In [13]:
def model_RF(x,y,estimadores, caracteristicas, impresion = False):

    tiempo_i = time.time()
    accuracy_list = np.zeros([4])
    precision_list = np.zeros([4,3])
    recall_list = np.zeros([4,3])
    f_list = np.zeros([4,3])
    errores = np.zeros(4)

    RF = RandomForestClassifier(n_estimators=estimadores, max_features =caracteristicas, n_jobs = -1)
    for j in range(4):
        Xtrain, Xtest, Ytrain, Ytest = train_test_split(x, y, test_size=0.25) # Modificar metodología de validación

        RF.fit(Xtrain, Ytrain)
        pred = RF.predict(Xtest)

        #code for calculating accuracy
        _accuracy_ = accuracy_score(Ytest, pred, normalize=True)
        accuracy_list[j] = _accuracy_

        #code for calculating recall
        _recalls_ = recall_score(Ytest, pred, average=None)
        recall_list[j] = _recalls_

        #code for calculating precision
        _precisions_ = precision_score(Ytest, pred, average=None)
        precision_list[j] = _precisions_

        _f_score_ = f1_score(Ytest, pred, average=None)
        f_list[j] = _f_score_


        errores[j] = classification_error(pred, Ytest)


    if impresion == True:
        #Curva ROC

        rf_probs = RF.predict_proba(Xtest)
        print(rf_probs)
        plot_roc(Xtest, Ytest, rf_probs, "Random Forest")
        auc = roc_auc_score(Ytest, pred)
        print('AUC: %.2f' % auc)
        fpr, tpr, thresholds = roc_curve(Ytest, pred)
        plot_roc_curve(fpr, tpr)
        #Matriz de confusión
        skplt.metrics.plot_confusion_matrix(Ytest, pred, normalize=True)

    return str(np.mean(accuracy_list)), str(np.std(accuracy_list)), str(np.mean(recall_list)), str(np.std(recall_list)), str(np.mean(precision_list)), str(np.std(precision_list)),  str(np.mean(f_list)), str(np.std(f_list)), str(np.mean(errores)), str(np.std(errores)), str(time.time()-tiempo_i)


Otra prueba con train test split, o sea que los datos no se dividen por periodos

In [14]:
otherDF=df.drop(['PERIODO'],axis=1)

In [21]:
y=otherDF['GRAVEDAD']
x=otherDF.drop(['GRAVEDAD'],axis=1)


In [17]:
Acc, IntAcc, Sen, IntSen, Pre, IntPre, f, IntF, error, stdError, tiempo = model_RF(x=x,y=y,estimadores=100,caracteristicas= 9, impresion = False)

In [18]:
# Métodos de sobremuestreo oversampling utilizar mediana en lugar y usar min max
print('Eficiencia',Acc, ' Int_Eficiencia', IntAcc,' Sensibilidad', Sen, ' Int_Sensibilidad',IntSen,' Precision', Pre, ' Int_Precision',IntPre,' F-Score', f, ' Int_F-Score',IntF,' Error_Prueba', error,' Int_Error', stdError,' Tiempo ejecución', tiempo)

Eficiencia 0.7506246700686259  Int_Eficiencia 0.0007649855751991657  Sensibilidad 0.5762942587381871  Int_Sensibilidad 0.2594819485136235  Precision 0.7291460350674016  Int_Precision 0.06435664084017832  F-Score 0.6098795298585189  Int_F-Score 0.2013471457391574  Error_Prueba 0.2493753299313743  Int_Error 0.000764985575199164  Tiempo ejecución 38.78389000892639


In [ ]:
import qgrid #Este paquete toca instalarlo o f
import pandas as pd
import numpy as np
pd.options.mode.chained_assignment = None
randn = np.random.randn
df_types = pd.DataFrame({
    'Numero de arboles' : pd.Series([5,5,5,5,5,
                                     10,10,10,10,10,
                                     20,20,20,20,20,
                                     50,50,50,50,50,
                                     100,100,100,100,100]),
    'Variables analizadas por nodo' : pd.Series([5,6,7,8,9,
                                                 5,6,7,8,9,
                                                 5,6,7,8,9,
                                                 5,6,7,8,9,
                                                 5,6,7,8,9])})
df_types["Eficiencia"] = ""
df_types["Int_Eficiencia"] = ""
df_types["Sensibilidad"] = ""
df_types["Int_Sensibilidad"] = ""
df_types["Precision"] = ""
df_types["Int_Precision"] = ""
df_types["F-Score"] = ""
df_types["Int_F-Score"] = ""

df_types["Error_Prueba"] = ""
df_types["Int_error"] = ""
df_types["Tiempo de ejecución"] = ""
df_types.set_index(['Numero de arboles','Variables analizadas por nodo'], inplace=True)

for n, k in df_types.index:
    Acc, IntAcc, Sen, IntSen, Pre, IntPre, f, IntF, error, stdError, tiempo = model_RF(x,y,n, k, impresion = False)
    df_types["Eficiencia"][n,k] = Acc
    df_types["Int_Eficiencia"][n,k] = IntAcc
    df_types["Sensibilidad"][n,k] = Sen
    df_types["Int_Sensibilidad"][n,k] = IntSen
    df_types["Precision"][n,k] = Pre
    df_types["Int_Precision"][n,k] = IntPre
    df_types["F-Score"][n,k] = f
    df_types["Int_F-Score"][n,k] = IntF
    df_types["Error_Prueba"][n,k] = error
    df_types["Int_error"][n,k] = stdError
    df_types["Tiempo de ejecución"][n,k] = tiempo


#df_types.sort_index(inplace=True)
qgrid_widget = qgrid.show_grid(df_types, show_toolbar=False)
qgrid_widget